In [2]:
import os
import re
import json

# Function to convert numeric strings to numbers
def convert_numbers(data):
    if isinstance(data, dict):
        return {k: convert_numbers(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_numbers(item) for item in data]
    elif isinstance(data, str) and data.isdigit():
        return int(data)
    elif isinstance(data, str):
        try:
            return float(data)
        except ValueError:
            return data
    return data

def config_to_dict(cfg_path: str, cfg_file: str):
    with open(os.path.join(cfg_path, cfg_file)) as f:
        data = f.read().split("\n")

    data = [line for line in data if "#" not in line and len(line) > 0]

    configs = {}
    for line in data:
        if line.startswith('-'):
            config_name, groups = line[1:].split()
            configs[config_name] = [group.split(':') for group in groups.split(',')]

    return convert_numbers(configs)

def dict_to_config(configs: dict, cfg_path: str, cfg_file: str):
    data = []
    for config_name in configs.keys():
        line = f"-{config_name} "
        item = configs[config_name]
        for i in range(len(item)):
            for j in range(len(item[i])):
                line += str(item[i][j])
                if j == len(item[i]) - 1:
                    break
                line += ":"
            
            if i == len(item) - 1:
                break
            line += ","
            
        data.append(line)
    
    if not os.path.exists(cfg_path):
        os.makedirs(cfg_path)
        
    with open(os.path.join(cfg_path, cfg_file), 'w') as f:
        for line in data:
            f.write(line + '\n')

In [6]:
import itertools
import numpy as np
import copy

def parse_range(string):
    if ':' in string:
        start, step, end = map(int, string.split(':'))
        return list(range(start, end + 1, step))
    if '*' in string:
        start, step, end = map(int, string.split('*'))
        values = []
        while start <= end:
            values.append(start)
            start *= step
        return values
    return []

def parse_structure(lst: list[str]):
    values = []
    if len(lst) > 0:
        if lst[0][0] == '=':
            vals = parse_range(lst[0][1:])
            values = [[val] * len(lst) for val in vals]
            
        if lst[0][0] == '|':
            individual_vals = []
            product = lst[0].split('|')[1]
            individual_vals.append(parse_range(lst[0].split('|')[-1]))
            for i in range(len(lst) - 2):
                individual_vals.append(parse_range(lst[i + 1].split('|')[-1]))
                
            if ':' in product or '*' in product:
                products = parse_range(product)
                for p in products:
                    values += [list(tup) + [int(p / np.prod(tup))]
                        for tup in list(itertools.product(*individual_vals))
                        if np.prod(tup) < p]
            else:
                product = int(product)
                values = [list(tup) + [int(product / np.prod(tup))]
                        for tup in list(itertools.product(*individual_vals))
                        if np.prod(tup) < product]
    
    return values

def parse_deltas(deltas: dict):
    count = 0
    p_deltas = {}
    for config_name in deltas.keys():
        for i in range(len(deltas[config_name])):
            structure = []
            for j in range(len(deltas[config_name][i])):
                item = deltas[config_name][i][j]
                if isinstance(item, str):
                    results = None
                    if item[0] == '|' or item[0] == '=':
                        structure.append(item)
                        if j == len(deltas[config_name][i]) - 1 or \
                            (deltas[config_name][i][j + 1][0] != '|' and
                            deltas[config_name][i][j + 1][0] != '='):
                            results = parse_structure(structure)
                            count += len(results)
                            structure = []
                            
                    elif ':' in item or '*' in item:
                        if len(structure) > 0:
                            structure = []
                        results = [[val] for val in parse_range(item)]
                        count += len(results)
                    
                    if results is not None:
                        if config_name not in p_deltas.keys():
                            p_deltas[config_name] = []
                        size = len(results[0])
                        p_deltas[config_name].append([
                            i,
                            list(range(j - size + 1, j + 1)),
                            results
                        ])
    
    print(f"Total: {count} configs")
    return p_deltas

def apply_parsed_deltas(configs: dict, p_deltas: dict, cfg_dlt_path: str):
    index = 0
    for config_name in p_deltas.keys():
        for entry in p_deltas[config_name]:
            i = entry[0]
            original_vals = copy.deepcopy(configs[config_name][i])
            for vals in entry[2]:
                for j in range(len(entry[1])):
                    configs[config_name][i][entry[1][j]] = vals[j]
                dict_to_config(configs, cfg_dlt_path, f"{str(index).zfill(4)}.config")
                index += 1
                
            configs[config_name][i] = original_vals
            
            


Total: 273 configs


In [7]:
cfg_path = "util/tuner/RTX2080_S"
cfg_file = "gpgpusim.config"
jsn_file = "gpgpusim.json"
configs = config_to_dict(cfg_path, cfg_file)

dlt_file = "deltas.json"
with open(os.path.join(cfg_path, dlt_file),) as f:
    deltas = json.load(f)
p_deltas = parse_deltas(deltas)
p_dlt_file = "parsed_deltas.json"
with open(os.path.join(cfg_path, p_dlt_file), 'w') as f:
    json.dump(p_deltas, f, indent=4)
apply_parsed_deltas(configs, p_deltas, os.path.join(cfg_path, "dlt"))

Total: 150 configs


In [9]:
# cfg_path = "gpu-simulator/gpgpu-sim/configs/tested-cfgs/RTX2080_S"
cfg_path = "util/tuner/RTX2080_S"
cfg_file = "gpgpusim.config"
jsn_file = "gpgpusim.json"
configs = config_to_dict(cfg_path, cfg_file)
with open(os.path.join(cfg_path, jsn_file), 'w') as f:
    json.dump(configs, f, indent=4)
# print(json.dumps(configs, indent=4))
# data = dict_to_config(configs, cfg_path, "out." + cfg_file)
# print(data)

dlt_file = "deltas_cuDGT_test2.json"
with open(os.path.join(cfg_path, dlt_file),) as f:
    deltas = json.load(f)
p_deltas = parse_deltas(deltas)
p_dlt_file = "parsed_deltas_cuDGT_test2.json"
with open(os.path.join(cfg_path, p_dlt_file), 'w') as f:
    json.dump(p_deltas, f, indent=4)
apply_parsed_deltas(configs, p_deltas, os.path.join(cfg_path, "dlt_cuDGT_test2"))

Total: 273 configs
